In [220]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [221]:
# Load your dataset
data = pd.read_csv("mt.csv")

# Separate input features and target values
X = data[['F1', 'F2']].values
y_class = data['T1'].values.reshape(-1, 1)
y_reg = data['T2'].values.reshape(-1, 1)

# Split your dataset into training and validation
X_train, X_val, y_train_class, y_val_class, y_train_reg, y_val_reg = train_test_split(X, y_class, y_reg, test_size=0.2, random_state=42)

# Train the multi-task neural network
input_size = 2
hidden_sizes = [10, 5]
output_sizes = [1, 1]
learning_rate = 0.01
epochs = 100

In [222]:
# sigmoid function
def sigmoid(x):
    exp = np.exp(-x)
    return 1 / (1 + exp)

In [223]:
# derivative of sigmoid function
def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

In [224]:
# hyperbolic tangent function
def tanh(x):
    return np.tanh(x)

In [225]:
# derivative of hyperbolic tangent function
def tanh_derivative(x):
    return 1 - np.tanh(x)**2

In [226]:
# initialize weights and biases
def initialize_weights(input_size, hidden_sizes, output_sizes):
    weights = [np.random.randn(input_size, hidden_sizes[0]), np.random.randn(hidden_sizes[0], hidden_sizes[1]), np.random.randn(hidden_sizes[1], output_sizes[0]), np.random.randn(hidden_sizes[1], output_sizes[1])]
    biases = [np.zeros((1, hidden_sizes[0])), np.zeros((1, hidden_sizes[1])), np.zeros((1, output_sizes[0])), np.zeros((1, output_sizes[1]))]
    return weights, biases

In [227]:
# forward propagation function
def forward_propogation(x, weights, biases):
    # Compute the linear combinations of inputs and weights for the first layer
    linear_combination_layer_1 = np.dot(x, weights[0]) + biases[0]
    # Apply the sigmoid activation function to the first layer output
    activation_layer_1 = sigmoid(linear_combination_layer_1)
    # Compute the linear combinations of first layer activations and weights for the second layer
    linear_combination_layer_2 = np.dot(activation_layer_1, weights[1]) + biases[1]
    # Apply the tanh activation function to the second layer output
    activation_layer_2 = tanh(linear_combination_layer_2)
    # Compute the linear combinations of second layer activations and weights for the output layer - classification task
    linear_combination_classification = np.dot(activation_layer_2, weights[2]) + biases[2]
    # Compute the linear combinations of second layer activations and weights for the output layer - regression task
    linear_combination_regression = np.dot(activation_layer_2, weights[3]) + biases[3]
    return linear_combination_layer_1, activation_layer_1, linear_combination_layer_2, activation_layer_2, linear_combination_classification, linear_combination_regression

In [228]:
def backward_propogation(weights, biases, learning_rate, inputs, y_class, y_reg, z_hidden_1, a_hidden_1, z_hidden_2, a_hidden_2, z_output_class, z_output_reg):
    # the number of samples in the dataset
    num_samples = y_class.shape[0]
    # compute the derivative of the loss function with respect to the output of the network
    delta_output_class = sigmoid(z_output_class) - y_class
    delta_output_reg = z_output_reg - y_reg
    # update the output layer weights and biases for its two tasks
    gradient_weights_output_class = np.dot(a_hidden_2.T, delta_output_class)
    gradient_biases_output_class = np.sum(delta_output_class, axis=0, keepdims=True)
    weights[2] -= learning_rate * gradient_weights_output_class
    biases[2] -= learning_rate * gradient_biases_output_class
    # derivative of the loss function with respect to the output of the network
    gradient_weights_output_reg = np.dot(a_hidden_2.T, delta_output_reg)
    gradient_biases_output_reg = np.sum(delta_output_reg, axis=0, keepdims=True)
    weights[3] -= learning_rate * gradient_weights_output_reg
    biases[3] -= learning_rate * gradient_biases_output_reg
    # computing the derivative of the loss function with respect to the output of the second hidden layer
    delta_hidden_2 = np.dot(delta_output_class, weights[2].T) + np.dot(delta_output_reg, weights[3].T)
    delta_hidden_2 *= tanh_derivative(z_hidden_2)
    # updating the weights and biases of the second hidden layer
    gradient_weights_hidden_2 = np.dot(a_hidden_1.T, delta_hidden_2)
    gradient_biases_hidden_2 = np.sum(delta_hidden_2, axis=0, keepdims=True)
    weights[1] -= learning_rate * gradient_weights_hidden_2
    biases[1] -= learning_rate * gradient_biases_hidden_2
    # computing the derivative of the loss function with respect to the output of the first hidden layer
    delta_hidden_1 = np.dot(delta_hidden_2, weights[1].T)
    delta_hidden_1 *= sigmoid_derivative(z_hidden_1)
    # updating the weights and biases of the first hidden layer
    gradient_weights_hidden_1 = np.dot(inputs.T, delta_hidden_1)
    gradient_biases_hidden_1 = np.sum(delta_hidden_1, axis=0, keepdims=True)
    weights[0] -= learning_rate * gradient_weights_hidden_1
    biases[0] -= learning_rate * gradient_biases_hidden_1
    # return the updated weights and biases
    return weights, biases

In [229]:
def train_multi_task_nn(X_train, y_train_class, y_train_reg, input_size, hidden_sizes, output_sizes, learning_rate, epochs):
    # Initialize weights and biases
    weights = []
    biases = []
    sizes = [input_size] + hidden_sizes + output_sizes
    # Randomly initialize weights and biases for each layer
    weights, biases = initialize_weights(input_size, hidden_sizes, output_sizes)
    # Train the model for the given number of epochs
    for epoch in range(epochs):
        # Forward propagation to get the predicted outputs
        z1, a1, z2, a2, z3_class, z3_reg = forward_propogation(X_train, weights, biases)
        # Backward propagation to update the weights and biases
        weights, biases = backward_propogation(weights, biases, learning_rate, X_train, y_train_class, y_train_reg, z1, a1, z2, a2, z3_class, z3_reg)
    # Return the trained weights and biases
    return weights, biases

In [230]:
def predict(weights, biases, X_test):
    # Forward propagation to get the predicted outputs
    z1, a1, z2, a2, z3_class, z3_reg = forward_propogation(X_test, weights, biases)
    # Return the predicted outputs
    return sigmoid(z3_class), z3_reg

In [231]:
# Calculate the accuracy of the model
def calculate_classification_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)
# Calculate the error of the model
def calculate_regression_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

In [232]:
Neural_network_model = train_multi_task_nn(X_train, y_train_class, y_train_reg, input_size, hidden_sizes, output_sizes, learning_rate, epochs)
y_pred_class, y_pred_reg = predict(Neural_network_model[0], Neural_network_model[1], X_val)

In [233]:
# Evaluate predictions
classification_accuracy = calculate_classification_accuracy(y_val_class, y_pred_class)
regression_error = calculate_regression_error(y_val_reg, y_pred_reg)

In [234]:
# printing the results of the model 
print("classification_accuracy: ", classification_accuracy)
print("regression_error: ", regression_error)

classification_accuracy:  0.3333333333333333
regression_error:  4.422727809606345e+159
